In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from models import DANN_Model

In [3]:
x_train_mnist = np.load('../data/mnist/x_train.npy')
y_train_mnist = np.load('../data/mnist/y_train.npy')

x_test_mnist = np.load('../data/mnist/x_test.npy')
y_test_mnist = np.load('../data/mnist/y_test.npy')

In [4]:
x_train_svhn = np.load('../data/svhn/x_train.npy')
y_train_svhn = np.load('../data/svhn/y_train.npy')

x_test_svhn = np.load('../data/svhn/x_test.npy')
y_test_svhn = np.load('../data/svhn/y_test.npy')

In [5]:
x_train_mnist, x_test_mnist = x_train_mnist / 255.0, x_test_mnist / 255.0
x_train_svhn, x_test_svhn = x_train_svhn / 255.0, x_test_svhn / 255.0

In [6]:
x_train_mnist = tf.cast(x_train_mnist, tf.float32)
x_test_mnist = tf.cast(x_test_mnist, tf.float32)
x_train_svhn = tf.cast(x_train_svhn, tf.float32)
x_test_svhn = tf.cast(x_test_svhn, tf.float32)

In [7]:
def pad_image(x, y):
    
    paddings = tf.constant([[2, 2,], [2, 2]])
    
    new_x = tf.pad(x, paddings, "CONSTANT")
    
    return (new_x, y)

def duplicate_channel(x, y):

    new_x = tf.stack([x, x, x], axis = -1)
    
    return (new_x, y)

In [8]:
mnist_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_mnist, y_train_mnist))
mnist_train_dataset = mnist_train_dataset.map(pad_image)
mnist_train_dataset = mnist_train_dataset.map(duplicate_channel)
target_train_dataset = mnist_train_dataset.shuffle(len(y_train_mnist))

mnist_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_mnist, y_test_mnist))
mnist_test_dataset = mnist_test_dataset.map(pad_image)
mnist_test_dataset = mnist_test_dataset.map(duplicate_channel)
target_test_dataset = mnist_test_dataset.shuffle(len(y_test_mnist))

svhn_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_svhn, y_train_svhn))
source_train_dataset = svhn_train_dataset.shuffle(len(y_train_svhn))

svhn_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_svhn, y_test_svhn))
source_test_dataset = svhn_train_dataset.shuffle(len(y_test_svhn))



source_train_dataset = source_train_dataset.batch(730)
source_train_dataset = source_train_dataset.prefetch(50)

source_test_dataset = source_test_dataset.batch(260)
source_test_dataset = source_test_dataset.prefetch(50)

target_train_dataset = target_train_dataset.batch(600)
target_train_dataset = target_train_dataset.prefetch(50)

# target_test_dataset = target_test_dataset.batch(500)
# target_test_dataset = target_test_dataset.prefetch(50)

In [9]:
model = DANN_Model(input_shape=(32, 32, 3), model_type='MNIST')

In [11]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for (source_images, class_labels), (target_images, _) in zip(source_train_dataset, target_train_dataset):
        model.train(source_images, class_labels, target_images)

    for (test_images, test_labels), (target_images, target_labels) in zip(source_test_dataset, target_train_dataset):
        model.test(test_images, test_labels, target_images, target_labels)
    
    print('Epoch: {}'.format(epoch + 1))
    print(model.log())
    model.reset_metrics()

Epoch: 1
L1: 0.2939, Acc1: 91.41, L1 Test: 0.2949, Acc1 Test: 91.35
L2: 0.0251, Acc2: 99.46, L2 Test: 0.0174, Acc2 Test: 99.60
L3 Test: 2.3399, Acc3 Test: 57.03

Epoch: 2
L1: 0.2761, Acc1: 92.01, L1 Test: 0.2782, Acc1 Test: 91.74
L2: 0.0202, Acc2: 99.56, L2 Test: 0.0116, Acc2 Test: 99.76
L3 Test: 2.3027, Acc3 Test: 56.95

Epoch: 3
L1: 0.2621, Acc1: 92.47, L1 Test: 0.2705, Acc1 Test: 91.92
L2: 0.0155, Acc2: 99.65, L2 Test: 0.0112, Acc2 Test: 99.73
L3 Test: 2.4210, Acc3 Test: 56.01

Epoch: 4
L1: 0.2507, Acc1: 92.81, L1 Test: 0.2601, Acc1 Test: 92.28
L2: 0.0178, Acc2: 99.55, L2 Test: 0.0145, Acc2 Test: 99.62
L3 Test: 2.3790, Acc3 Test: 57.08

Epoch: 5
L1: 0.2401, Acc1: 93.09, L1 Test: 0.2425, Acc1 Test: 92.71
L2: 0.0199, Acc2: 99.48, L2 Test: 0.0124, Acc2 Test: 99.71
L3 Test: 2.3652, Acc3 Test: 57.79

Epoch: 6
L1: 0.2339, Acc1: 93.21, L1 Test: 0.2289, Acc1 Test: 93.15
L2: 0.0151, Acc2: 99.60, L2 Test: 0.0076, Acc2 Test: 99.83
L3 Test: 2.5497, Acc3 Test: 57.49

Epoch: 7
L1: 0.2234, Acc1: 9